In [1]:
import requests
from requests.auth import HTTPBasicAuth

def generarToken(usuario):
    url = "https://apisolicitudesonlinepre.cinconet.local/api/Auth/login"
    username = usuario
    password = "tu_contraseña"
    response = requests.post(url, auth=HTTPBasicAuth(username, password), verify=False)
    return response.json().get("validToken")

In [12]:
import requests

mail="vchumillas@megamedia.es"
print(mail)
usuario, dominio = mail.split('@')

print("Usuario:", usuario)
print("Dominio:", dominio)

validToken = generarToken(mail)
url = f"https://apisolicitudesonline.cinconet.local/api/Gestion/Usuario/Directorio/{usuario}"
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'Authorization': f'Bearer {validToken}'
}

response = requests.get(url, headers=headers, verify=False)
print(response.text)


vchumillas@megamedia.es
Usuario: vchumillas
Dominio: megamedia.es


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonlinepre.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"mail":"vchumillas@megamedia.es","mail_2":"vchumillas@megamedia.es","clasificacionProfesional":"TEC","rolId":0,"rol":null,"autorizador":null,"activo":false,"correo":null,"empresa":"MEGAMEDIA TELEVISION, S.L.","empresaId":31,"departamento":"DESARROLLO, PRODUCTO E INNOVACION","localizacion":"Edificio B Planta 4","numeroEmpleado":310536,"telefono":null,"esAutorizador":false,"delegadosId":null,"delegados":null,"isApprover":false,"canDelegate":false,"esAuditor":false,"esAdministrado":false,"id":0,"login":"vchumillas","nombre":"VICTOR","apellidos":"CHUMILLAS HERNANDO"}


c:\Users\vchumillas\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apisolicitudesonline.cinconet.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
